### Notebook da eseguire solo dopo SchemaAlignment.ipynb

In [2]:
import py_entitymatching as em
import pandas as pd
import os, sys

/Users/mauri/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# DataProfiling

In [3]:
final = pd.read_csv("./src/final_dataset.csv")

In [5]:
final.dtype

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [66]:
final.rename(columns={"Unnamed: 0": "ID"}, inplace=True)

In [67]:
final.head()

,ID,name,country,sector,founded,marketcap,revenue,employees,links,ceo
0,0.0,LACEWORK,UNITED STATES,CYBERSECURITY,2015.0,NaN,NaN,NaN,NaN,NaN
1,1.0,TIPALTI,UNITED STATES,FINTECH,2010.0,NaN,NaN,NaN,NaN,NaN
2,2.0,TEMPUS,UNITED STATES,HEALTH,2015.0,NaN,NaN,NaN,NaN,NaN
3,3.0,ANDURIL,UNITED STATES,ARTIFICIAL INTELLIGENCE,2017.0,NaN,NaN,NaN,NaN,NaN
4,4.0,BOLT,ESTONIA,AUTO & TRANSPORTATION,2013.0,NaN,NaN,NaN,NaN,NaN


La lettura del file csv ("./src/final_dataset.csv") porta a un errore. La 42185-esima riga presenta un ID nullo. Questo rende l'attributo ID non adatto a funzionare da chiave per le operazioni successive riguardanti il blocking. La chiave non deve avere valori nulli. Eliminiamo la riga.

In [68]:
final[final['ID'].isna()]

,ID,name,country,sector,founded,marketcap,revenue,employees,links,ceo
42184,NaN,USA,NaN,NaN,2.880000e+09,NaN,NaN,NaN,NaN,NaN


In [69]:
final.drop(42184,axis=0,inplace=True)
final[final['ID'].isna()]

,ID,name,country,sector,founded,marketcap,revenue,employees,links,ceo


In [70]:
final.to_csv("./src/final_dataset.csv", index = False)

Testiamo un ulteriore lettura del file e verifichiamo se l'attributo ID (si mostra come Unnamed: 0) è UNIQUE e UNIFORM tramite DataProfiling

In [73]:
final = pd.read_csv("./src/final_dataset.csv")
final.head()

,ID,name,country,sector,founded,marketcap,revenue,employees,links,ceo
0,0.0,LACEWORK,UNITED STATES,CYBERSECURITY,2015.0,NaN,NaN,NaN,NaN,NaN
1,1.0,TIPALTI,UNITED STATES,FINTECH,2010.0,NaN,NaN,NaN,NaN,NaN
2,2.0,TEMPUS,UNITED STATES,HEALTH,2015.0,NaN,NaN,NaN,NaN,NaN
3,3.0,ANDURIL,UNITED STATES,ARTIFICIAL INTELLIGENCE,2017.0,NaN,NaN,NaN,NaN,NaN
4,4.0,BOLT,ESTONIA,AUTO & TRANSPORTATION,2013.0,NaN,NaN,NaN,NaN,NaN


In [72]:
from pandas_profiling import ProfileReport
profile=ProfileReport(final_dataset)
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# DownSampling

Necessario per testare il blocking su campioni di dati

In [80]:
sample_A, sample_B = em.down_sample(final, final, size=1000, y_param=1, show_progress=False)

In [81]:
sample_A.head()

,ID,name,country,sector,founded,marketcap,revenue,employees,links,ceo
0,0.0,LACEWORK,UNITED STATES,CYBERSECURITY,2015.0,NaN,NaN,NaN,NaN,NaN
26625,26625.0,EMIRATES NBD,UAE,NaN,NaN,2.390000e+10,NaN,NaN,NaN,AHMED BIN SAEED AL MAKTOUM
30722,30722.0,SISTEMA PUBLIC JOINT STOCK FINANCIAL CORPORATION,RUSSIA,NaN,NaN,3.869000e+09,NaN,NaN,NaN,NaN
34819,34819.0,PT BENTOEL INTERNASIONAL INVESTAMA TBK,INDONESIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6145,6145.0,SHUTTERSTOCK,USA,TECH INTERNET,NaN,2.230000e+09,NaN,NaN,NaN,NaN


In [82]:
sample_B.head()

,ID,name,country,sector,founded,marketcap,revenue,employees,links,ceo
168206,168206.0,P. ALLEN LIMITED,70MARKETSTREET TOTTINGTON BURY BL83LJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64176,64176.0,TELUS INTERNATIONAL,CANADA,NaN,NaN,5.980000e+09,NaN,NaN,NaN,NaN
108182,108182.0,ONE,SINGAPORE,INTERNET SOFTWARE & SERVICES,NaN,NaN,NaN,NaN,NaN,NaN
131901,131901.0,ACHIKO AG,SWITZERLAND,NaN,NaN,NaN,NaN,29.0,NaN,MR. WERN-YI GOH
169338,169338.0,TOWER SPA,ITALY,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Blocking
## Metadata da estrarre (bisogna trovare il modo)

In [77]:
ob = em.OverlapBlocker()

In [78]:
A = em.read_csv_metadata("./src/final_dataset.csv", key="ID")

Metadata file is not present in the given path; proceeding to read the csv file.


In [83]:
C1 = ob.block_tables(sample_A, sample_B, 'name', 'name', word_level=False, q_val=3, overlap_size=3, 
                    l_output_attrs=['ID', 'name', 'country'], 
                    r_output_attrs=['ID', 'name', 'country'],
                    show_progress=False)

DataFrame information is not present in the catalog


KeyError: 'DataFrame information is not present in the catalog'

In [79]:
C2 = ob.block_candset(A, 'name', 'name', word_level=False,
                      q_val= 3, overlap_size=1, show_progress=False)

Requested metadata ( fk_ltable ) for the given DataFrame is not present in the catalog


KeyError: 'Requested metadata ( fk_ltable ) for the given DataFrame is not present in the catalog'